In [1]:
import secrets
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes
from hashlib import sha256

# Diffie-Hellman Set up


1. A and B agree on two large prime numbers 𝑝 and 𝑔, with 𝑔 a primitive root of 𝑝. We will choose a group from https://datatracker.ietf.org/doc/html/rfc3526#section-3. which has predefined primes and generators that have been researched before and proven secure. For demo purposes we will choose the 2048-bit MODP Group


In [2]:
# Use standardized 2048-bit MODP group from RFC 3526 (Group 14)

p_value = int(
    """FFFFFFFF FFFFFFFF C90FDAA2 2168C234 C4C6628B 80DC1CD1
      29024E08 8A67CC74 020BBEA6 3B139B22 514A0879 8E3404DD
      EF9519B3 CD3A431B 302B0A6D F25F1437 4FE1356D 6D51C245
      E485B576 625E7EC6 F44C42E9 A637ED6B 0BFF5CB6 F406B7ED
      EE386BFB 5A899FA5 AE9F2411 7C4B1FE6 49286651 ECE45B3D
      C2007CB8 A163BF05 98DA4836 1C55D39A 69163FA8 FD24CF5F
      83655D23 DCA3AD96 1C62F356 208552BB 9ED52907 7096966D
      670C354E 4ABC9804 F1746C08 CA18217C 32905E46 2E36CE3B
      E39E772C 180E8603 9B2783A2 EC07A28F B5C55DF0 6F4C52C9
      DE2BCBF6 95581718 3995497C EA956AE5 15D22618 98FA0510
      15728E5A 8AACAA68 FFFFFFFF FFFFFFFF""".replace("\n", "").replace(" ", ""), 16
)

g_value = 2  # always 2 for MODP groups

print("Using standardized DH group (2048-bit MODP from RFC 3526)")
print("p =", p_value)
print("g =", g_value)


Using standardized DH group (2048-bit MODP from RFC 3526)
p = 32317006071311007300338913926423828248817941241140239112842009751400741706634354222619689417363569347117901737909704191754605873209195028853758986185622153212175412514901774520270235796078236248884246189477587641105928646099411723245426622522193230540919037680524235519125679715870117001058055877651038861847280257976054903569732561526167081339361799541336476559160368317896729073178384589680639671900977202194168647225871031411336429319536193471636533209717077448227988588565369208645296636077250268955505928362751121174096972998068410554359584866583291642136218231078990999448652468262416972035911852507045361090559
g = 2


2. Both A and B generate a random number (say 𝑎 and 𝑏). Those need to be kept secret!! a and b ∈ {2,..., p−2} (from textbook)

In [3]:
def generate_secret(p):
    """
    Generate a random secret integer in [2, p-2].
    """
    return secrets.randbelow(p - 3) + 2  # shift range to [2, p-2]

alice_secret = generate_secret(p_value)
bob_secret   = generate_secret(p_value)

print("Alice's secret (a):", alice_secret)
print("Bob's secret (b):", bob_secret)

Alice's secret (a): 15188519838151563089809258819939049125863287540520910568239553665956354712499758874736849368404589865085844283530513772472037267548701161090499207131182312382515470375263884448508627836765586260760411485903573256913556300011848238894714849178144883282585483690260767853676308575216820204926195020382459070871286536982582594568151352414338071907499208817444789667462773624689616470488456165804353364544614790052374027339199997770398196069849784485842160091596259551916675982857602962859797351520928796149187997387877058939463524246797068971834225724125235513569946370803904775868671655455440024181762947196345023079416
Bob's secret (b): 27324270494723139917545969027346035596819970112421626763558922532888501310124544522184817547206670357551115652498937092479185808500997764739814513324427508441495674911584498509604416541899391572036566061399096733857184830203741708663198838432503219218681687288455843281461270938352587266368237251175381739631204948788967820474461298807983395464

3. A sends 𝑔^𝑎 mod 𝑝 to B via a public insecure channel
4. B sends 𝑔^𝑏 mod 𝑝 to A via a public insecure channel

In [4]:
alice_public = pow(g_value, alice_secret, p_value)  # g^a mod p
bob_public   = pow(g_value, bob_secret, p_value)    # g^b mod p

print("Alice's public value (A):", alice_public)
print("Bob's public value (B):", bob_public)

Alice's public value (A): 12226421392341105792714356474673278167761109865147727885100846727610932832726568753400378933502297284821357200309266090818518542706609012324334962219301765259993113105716524432362869926686026402603888278839762723475182257905485325116284718069447122231462937020882975564382997322661376750213721802682579193834251411722332696212051271443076287921312755879808952697338268153764934605467603945205317113495971439422390155157370775661297720268272341581252492963968536492129419985466911443153949120516746244523511315418541885019158323993555686761954519531949608838829633020128174968464637041087697101704035343533751539816210
Bob's public value (B): 22802843769667804375555148504328357010277832331280309986578962700936122413088581143412131163349385445863566176097159356092710050523013904614193052678327126516526508347485858294442062274377512473802208703340763714958933912774449911996722367921814715877228442348905496374857503107606288972064696413363358452904454059454985320889806256

5. A calculates (𝑔^𝑏 )^𝑎≡𝑔^𝑎𝑏 mod 𝑝 and 𝐵 calculates (𝑔^𝑎 )^𝑏≡𝑔^𝑎𝑏 mod 𝑝. The result can now be used as a shared secret key 

In [5]:
alice_shared = pow(bob_public, alice_secret, p_value)  # (g^b)^a mod p
bob_shared   = pow(alice_public, bob_secret, p_value)  # (g^a)^b mod p

print("Alice's shared secret:", alice_shared)
print("Bob's shared secret:  ", bob_shared)
print("Do they match?        ", alice_shared == bob_shared)

Alice's shared secret: 5666617959676088018986146308367910139042152310581286770658068885142133250560398821231426196128170775359982849322015140632538688254210538616716920399355904630876387134404824987766196239748582256929139178371293305567212225465859908722102741114156983036715393416930390663172612824389965491704261086187870173192458562400337666933463407868413176907894511943410937050771315477774535118999225921574614446421990550507629073195859849768323195339150150351650271151674516684397851393650296460166269588811918211436827881430050059650511581713359609002230922570913962691168181053930661036608857543306509289244447354912087236381022
Bob's shared secret:   5666617959676088018986146308367910139042152310581286770658068885142133250560398821231426196128170775359982849322015140632538688254210538616716920399355904630876387134404824987766196239748582256929139178371293305567212225465859908722102741114156983036715393416930390663172612824389965491704261086187870173192458562400337666933463407868413

In [6]:
shared_key = sha256(alice_shared.to_bytes((alice_shared.bit_length() + 7) // 8, 'big')).digest()
print("Derived 256-bit AES key:", shared_key.hex())

Derived 256-bit AES key: 03b462a81e3a1b951678d755b9ed7960ae0e36439221d8fc82cf8f1aeeda7caf


# AES

In [7]:
from base64 import b64encode


data = 'Lorem ipsum dolor sit amet, consectetur adipiscing elit,' \
'sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. ' \
'Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ' \
'ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate ' \
'velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, ' \
'sunt in culpa qui officia deserunt mollit anim id est laborum.'.encode('utf_8')

custom_iv = get_random_bytes(AES.block_size)

In [8]:
def encypt_AES(plaintext, key, iv):
    cipher = AES.new(key, AES.MODE_CBC, iv=custom_iv)
    ct_bytes = cipher.encrypt(pad(plaintext, AES.block_size)) 
    initial_vector = b64encode(cipher.iv).decode('utf-8') #initialization vector -> probabilistic method
    ct = b64encode(ct_bytes).decode('utf-8') #cipher text
    return initial_vector, ct


iv, ct = encypt_AES(plaintext=data, key=shared_key, iv=custom_iv)
print(f"ciphertext: {ct}")

ciphertext: M+iAkS3TFWwmdUL+EfUEDytwcjwVKMwypanf81vm4ukVoMAYhPnBMjnG5HkhAVRiZt6M9GG/9hx2C4ErpCnJ7YNRIJDnVUNAa6ZhAIRCu76xoWq1EvTJgnCj/vJ/ORu6AtKP4TaRwc7vhPLh7Ulvde2oqdG8iAZn45iajhmGoJ8Slsk11XSz1mKN39KDSXCpmOoQRvLz65r7DQWsvGojDQwhxgMmnuLuY9iGSDP5HhYJYj9C7rSCyis4VcJBhuc9U2Yo2YwtcGeIrikHZGW6I1Ln+kgMU3uULWi/y/GBqZDElh7LLf3ViO8BZQ4jSoNPJ0F3pnxYkJIfQ+cwxUhS0iPjvZLV2UdqCuT6wFw9qE2aZdfE+N5wKKBVwfSlYlI6Y83nDQOMtKhlBwzLQ7Ez80UNQS5UedDUkIo8BL4mmOywGrgxaXILxb1E53ch0SFVQuoDiF9ASsYmBcS3CRdftEe4vaGmwfncKAl59jKUgQmt55WnQpnm70Q+2DQAuzqpNVvHE44g8PeipJBqndX1R1VDoXY/vhW1i9E+7KaeZIRb99AGh4ADLgnUxx25NU+3agEojRcvQ9S/Xs1qaVKpnw==


In [9]:
from base64 import b64decode

def decrypt_AES(iv, ct, key):
    try:
        cipher = AES.new(key, AES.MODE_CBC, iv= b64decode(iv))
        pt = unpad(cipher.decrypt(b64decode(ct)), AES.block_size)
        print("The message was: ", pt)
    except (ValueError, KeyError):
        print("Incorrect decryption")

decrypt_AES(iv, ct, shared_key)

The message was:  b'Lorem ipsum dolor sit amet, consectetur adipiscing elit,sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.'
